In [1]:
import librosa
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os

In [ ]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)  # Average MFCCs for now

In [ ]:
def load_data(data_dir):
    singer_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    features, labels = [], []
    for i, singer in enumerate(singer_names):
        folder_path = os.path.join(data_dir, singer)
        print(f"Loading {singer} samples...")
        for audio_file in os.listdir(folder_path):
            if audio_file.endswith('.wav'):
                file_path = os.path.join(folder_path, audio_file)
                voice_features = extract_features(file_path)
                features.append(voice_features)
                labels.append(i)  # Use integer labels
    return np.array(features), np.array(labels), singer_names

In [ ]:

data_dir = '/home/nirjhar/CODE/greeneggsandham/cookiepuss5'

In [ ]:
print("Loading voice samples...")
X, y, singer_names = load_data(data_dir)
print(f"Total samples: {len(X)}")
print(f"Features per sample: {X.shape[1]}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("Tuning SVM hyperparameters...")
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}
svm_model = svm.SVC(decision_function_shape='ovr')
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV accuracy: {grid_search.best_score_:.2f}")

In [ ]:
model = grid_search.best_estimator_
print("Training the SVM model...")
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f} ({accuracy*100:.1f}%)")

# Sample predictions
for i in range(min(5, len(X_test_scaled))):
    predicted = singer_names[y_pred[i]]
    actual = singer_names[y_test[i]]
    print(f"Actual: {actual}, Predicted: {predicted}")